In [1]:
##################
# Libraries
##################
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import LinearSVC, SVC
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from loguru import logger

In [2]:
##################
# Constants
##################
MODEL_SEED=42
ROWS_SEED=[24, 42, 206, 602, 412, 214, 754, 457, 2023, 3202]
SIZE_OF_UNLABELLED_DATA = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.85, 0.9]
MODELS = [
    GaussianNB(var_smoothing=0.01519911082952933),
    DecisionTreeClassifier(criterion='gini', max_depth=5, min_samples_leaf=1, min_samples_split=3),
    KNeighborsClassifier(metric='manhattan', n_neighbors=9, weights='distance'),
    RandomForestClassifier(criterion='gini', max_depth=11, min_samples_leaf=1, min_samples_split=2, n_estimators=100),
]

In [3]:
##################
# Functions
##################
def encode_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    """Encode dataframe using LabelEncoder"""
    labelencoder = LabelEncoder()

    for col in df.columns:
        df[col] = labelencoder.fit_transform(df[col])

    return df

def get_metrics() -> dict:
    metrics = {
        'accuracy': accuracy_score,
        'precision': precision_score,
        'recall': recall_score,
        'f1': f1_score,
        'roc': roc_auc_score
    }
    return metrics


def get_metrics_df(y_true, y_pred) -> pd.DataFrame:
    metrics = get_metrics()
    df = pd.DataFrame()
    for metric_name, metric in metrics.items():
        df[metric_name] = [metric(y_true, y_pred)]

    return df

def run_pipeline(X_train, X_test, y_train, y_test, size_of_unlabelled_data=0.5, model=None, rows_seed=42) -> dict:
    if model is None:
        raise ValueError("Model is None")
    
    X_train = X_train.copy()
    X_test = X_test.copy()
    y_train = y_train.copy()
    y_test = y_test.copy()

    # Unlabelled data
    rng = np.random.RandomState(rows_seed)
    random_rows_with_rng = rng.choice(X_train.index, size=int(len(X_train)*size_of_unlabelled_data), replace=False)
    y_train.loc[random_rows_with_rng] = -1

    # Scale data
    scaler = RobustScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)


    self_training_model = SelfTrainingClassifier(model, verbose=False)
    self_training_model.fit(X_train, y_train)

    # Predict
    y_pred = self_training_model.predict(X_test)

    # Evaluate - Get Metrics
    metrics = get_metrics_df(y_test, y_pred)
    accuracy = metrics["accuracy"].values[0]
    precision = metrics["precision"].values[0]
    recall = metrics["recall"].values[0]
    f1 = metrics["f1"].values[0]
    roc = metrics["roc"].values[0]
    

    return {"rows_seed": rows_seed, "model": type(model).__name__, "unlabeled": size_of_unlabelled_data, "acc": accuracy, "precision": precision, "recall": recall, "f1": f1, "roc": roc}

def run_pipeline_self(X_train, X_test, y_train, y_test) -> pd.DataFrame:
    run_counter = 0
    results = []

    for model in MODELS:
        for value in SIZE_OF_UNLABELLED_DATA:
            logger.info(f'Model: {model}')
            logger.info(f'Size of unlabelled data: {value}')
            for row_seed in ROWS_SEED:
                run_counter += 1
                logger.info(f"%{run_counter} - Running pipeline for row_seed: {row_seed} and size_of_unlabelled_data: {value} and model: {model}")
                result = run_pipeline(X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test, size_of_unlabelled_data=value, model=model, rows_seed=row_seed)
                results.append(result)
            print('\n')

    results_rows_seed = []
    results_models = []
    results_unlabeled = []
    results_acc = []
    results_precision = []
    results_recall = []
    results_f1 = []
    results_roc = []

    for result in results:
        results_rows_seed.append(result.get('rows_seed'))
        results_models.append(result.get('model'))
        results_unlabeled.append(result.get('unlabeled'))
        results_acc.append(result.get('acc'))
        results_precision.append(result.get('precision'))
        results_recall.append(result.get('recall'))
        results_f1.append(result.get('f1'))
        results_roc.append(result.get('roc'))


    results_df = pd.DataFrame({'rows_seed': results_rows_seed, 'model': results_models, 'unlabeled': results_unlabeled, 'acc': results_acc, 'precision': results_precision, 'recall': results_recall, 'f1': results_f1, 'roc': results_roc})

    return results_df

    

In [4]:
df = pd.read_excel('../../datasets/Pumpkin_Seeds_Dataset.xlsx')

labelencoder = LabelEncoder()
df['Class'] = labelencoder.fit_transform(df['Class'])


X = df.drop('Class', axis=1)
y = df['Class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=MODEL_SEED)

In [5]:
results_df = run_pipeline_self(X_train, X_test, y_train, y_test)

2023-10-22 14:25:46.791 | INFO     | __main__:run_pipeline_self:75 - Model: GaussianNB(var_smoothing=0.01519911082952933)
2023-10-22 14:25:46.792 | INFO     | __main__:run_pipeline_self:76 - Size of unlabelled data: 0.1
2023-10-22 14:25:46.792 | INFO     | __main__:run_pipeline_self:79 - %1 - Running pipeline for row_seed: 24 and size_of_unlabelled_data: 0.1 and model: GaussianNB(var_smoothing=0.01519911082952933)
2023-10-22 14:25:46.808 | INFO     | __main__:run_pipeline_self:79 - %2 - Running pipeline for row_seed: 42 and size_of_unlabelled_data: 0.1 and model: GaussianNB(var_smoothing=0.01519911082952933)
2023-10-22 14:25:46.822 | INFO     | __main__:run_pipeline_self:79 - %3 - Running pipeline for row_seed: 206 and size_of_unlabelled_data: 0.1 and model: GaussianNB(var_smoothing=0.01519911082952933)
2023-10-22 14:25:46.836 | INFO     | __main__:run_pipeline_self:79 - %4 - Running pipeline for row_seed: 602 and size_of_unlabelled_data: 0.1 and model: GaussianNB(var_smoothing=0.01519

2023-10-22 14:25:47.146 | INFO     | __main__:run_pipeline_self:79 - %25 - Running pipeline for row_seed: 412 and size_of_unlabelled_data: 0.3 and model: GaussianNB(var_smoothing=0.01519911082952933)
2023-10-22 14:25:47.164 | INFO     | __main__:run_pipeline_self:79 - %26 - Running pipeline for row_seed: 214 and size_of_unlabelled_data: 0.3 and model: GaussianNB(var_smoothing=0.01519911082952933)
2023-10-22 14:25:47.182 | INFO     | __main__:run_pipeline_self:79 - %27 - Running pipeline for row_seed: 754 and size_of_unlabelled_data: 0.3 and model: GaussianNB(var_smoothing=0.01519911082952933)
2023-10-22 14:25:47.197 | INFO     | __main__:run_pipeline_self:79 - %28 - Running pipeline for row_seed: 457 and size_of_unlabelled_data: 0.3 and model: GaussianNB(var_smoothing=0.01519911082952933)
2023-10-22 14:25:47.214 | INFO     | __main__:run_pipeline_self:79 - %29 - Running pipeline for row_seed: 2023 and size_of_unlabelled_data: 0.3 and model: GaussianNB(var_smoothing=0.01519911082952933)

2023-10-22 14:25:47.466 | INFO     | __main__:run_pipeline_self:79 - %45 - Running pipeline for row_seed: 412 and size_of_unlabelled_data: 0.5 and model: GaussianNB(var_smoothing=0.01519911082952933)
2023-10-22 14:25:47.487 | INFO     | __main__:run_pipeline_self:79 - %46 - Running pipeline for row_seed: 214 and size_of_unlabelled_data: 0.5 and model: GaussianNB(var_smoothing=0.01519911082952933)
2023-10-22 14:25:47.507 | INFO     | __main__:run_pipeline_self:79 - %47 - Running pipeline for row_seed: 754 and size_of_unlabelled_data: 0.5 and model: GaussianNB(var_smoothing=0.01519911082952933)
2023-10-22 14:25:47.525 | INFO     | __main__:run_pipeline_self:79 - %48 - Running pipeline for row_seed: 457 and size_of_unlabelled_data: 0.5 and model: GaussianNB(var_smoothing=0.01519911082952933)
2023-10-22 14:25:47.540 | INFO     | __main__:run_pipeline_self:79 - %49 - Running pipeline for row_seed: 2023 and size_of_unlabelled_data: 0.5 and model: GaussianNB(var_smoothing=0.01519911082952933)

2023-10-22 14:25:47.779 | INFO     | __main__:run_pipeline_self:79 - %64 - Running pipeline for row_seed: 602 and size_of_unlabelled_data: 0.7 and model: GaussianNB(var_smoothing=0.01519911082952933)
2023-10-22 14:25:47.796 | INFO     | __main__:run_pipeline_self:79 - %65 - Running pipeline for row_seed: 412 and size_of_unlabelled_data: 0.7 and model: GaussianNB(var_smoothing=0.01519911082952933)
2023-10-22 14:25:47.813 | INFO     | __main__:run_pipeline_self:79 - %66 - Running pipeline for row_seed: 214 and size_of_unlabelled_data: 0.7 and model: GaussianNB(var_smoothing=0.01519911082952933)
2023-10-22 14:25:47.835 | INFO     | __main__:run_pipeline_self:79 - %67 - Running pipeline for row_seed: 754 and size_of_unlabelled_data: 0.7 and model: GaussianNB(var_smoothing=0.01519911082952933)
2023-10-22 14:25:47.852 | INFO     | __main__:run_pipeline_self:79 - %68 - Running pipeline for row_seed: 457 and size_of_unlabelled_data: 0.7 and model: GaussianNB(var_smoothing=0.01519911082952933)


2023-10-22 14:25:48.115 | INFO     | __main__:run_pipeline_self:79 - %84 - Running pipeline for row_seed: 602 and size_of_unlabelled_data: 0.85 and model: GaussianNB(var_smoothing=0.01519911082952933)
2023-10-22 14:25:48.131 | INFO     | __main__:run_pipeline_self:79 - %85 - Running pipeline for row_seed: 412 and size_of_unlabelled_data: 0.85 and model: GaussianNB(var_smoothing=0.01519911082952933)
2023-10-22 14:25:48.147 | INFO     | __main__:run_pipeline_self:79 - %86 - Running pipeline for row_seed: 214 and size_of_unlabelled_data: 0.85 and model: GaussianNB(var_smoothing=0.01519911082952933)
2023-10-22 14:25:48.164 | INFO     | __main__:run_pipeline_self:79 - %87 - Running pipeline for row_seed: 754 and size_of_unlabelled_data: 0.85 and model: GaussianNB(var_smoothing=0.01519911082952933)
2023-10-22 14:25:48.180 | INFO     | __main__:run_pipeline_self:79 - %88 - Running pipeline for row_seed: 457 and size_of_unlabelled_data: 0.85 and model: GaussianNB(var_smoothing=0.01519911082952

...

2023-10-22 14:28:01.859 | INFO     | __main__:run_pipeline_self:79 - %352 - Running pipeline for row_seed: 42 and size_of_unlabelled_data: 0.6 and model: RandomForestClassifier(max_depth=11)
2023-10-22 14:28:04.667 | INFO     | __main__:run_pipeline_self:79 - %353 - Running pipeline for row_seed: 206 and size_of_unlabelled_data: 0.6 and model: RandomForestClassifier(max_depth=11)
2023-10-22 14:28:07.529 | INFO     | __main__:run_pipeline_self:79 - %354 - Running pipeline for row_seed: 602 and size_of_unlabelled_data: 0.6 and model: RandomForestClassifier(max_depth=11)
2023-10-22 14:28:10.283 | INFO     | __main__:run_pipeline_self:79 - %355 - Running pipeline for row_seed: 412 and size_of_unlabelled_data: 0.6 and model: RandomForestClassifier(max_depth=11)
2023-10-22 14:28:13.107 | INFO     | __main__:run_pipeline_self:79 - %356 - Running pipeline for row_seed: 214 and size_of_unlabelled_data: 0.6 and model: RandomForestClassifier(max_depth=11)
2023-10-22 14:28:15.813 | INFO     | __ma

2023-10-22 14:28:29.843 | INFO     | __main__:run_pipeline_self:79 - %362 - Running pipeline for row_seed: 42 and size_of_unlabelled_data: 0.7 and model: RandomForestClassifier(max_depth=11)
2023-10-22 14:28:32.583 | INFO     | __main__:run_pipeline_self:79 - %363 - Running pipeline for row_seed: 206 and size_of_unlabelled_data: 0.7 and model: RandomForestClassifier(max_depth=11)
2023-10-22 14:28:35.302 | INFO     | __main__:run_pipeline_self:79 - %364 - Running pipeline for row_seed: 602 and size_of_unlabelled_data: 0.7 and model: RandomForestClassifier(max_depth=11)
2023-10-22 14:28:37.981 | INFO     | __main__:run_pipeline_self:79 - %365 - Running pipeline for row_seed: 412 and size_of_unlabelled_data: 0.7 and model: RandomForestClassifier(max_depth=11)
2023-10-22 14:28:40.697 | INFO     | __main__:run_pipeline_self:79 - %366 - Running pipeline for row_seed: 214 and size_of_unlabelled_data: 0.7 and model: RandomForestClassifier(max_depth=11)
2023-10-22 14:28:43.305 | INFO     | __ma

2023-10-22 14:28:56.263 | INFO     | __main__:run_pipeline_self:79 - %372 - Running pipeline for row_seed: 42 and size_of_unlabelled_data: 0.8 and model: RandomForestClassifier(max_depth=11)
2023-10-22 14:28:58.722 | INFO     | __main__:run_pipeline_self:79 - %373 - Running pipeline for row_seed: 206 and size_of_unlabelled_data: 0.8 and model: RandomForestClassifier(max_depth=11)
2023-10-22 14:29:01.274 | INFO     | __main__:run_pipeline_self:79 - %374 - Running pipeline for row_seed: 602 and size_of_unlabelled_data: 0.8 and model: RandomForestClassifier(max_depth=11)
2023-10-22 14:29:03.765 | INFO     | __main__:run_pipeline_self:79 - %375 - Running pipeline for row_seed: 412 and size_of_unlabelled_data: 0.8 and model: RandomForestClassifier(max_depth=11)
2023-10-22 14:29:06.417 | INFO     | __main__:run_pipeline_self:79 - %376 - Running pipeline for row_seed: 214 and size_of_unlabelled_data: 0.8 and model: RandomForestClassifier(max_depth=11)
2023-10-22 14:29:08.713 | INFO     | __ma

2023-10-22 14:29:20.533 | INFO     | __main__:run_pipeline_self:79 - %382 - Running pipeline for row_seed: 42 and size_of_unlabelled_data: 0.85 and model: RandomForestClassifier(max_depth=11)
2023-10-22 14:29:22.896 | INFO     | __main__:run_pipeline_self:79 - %383 - Running pipeline for row_seed: 206 and size_of_unlabelled_data: 0.85 and model: RandomForestClassifier(max_depth=11)
2023-10-22 14:29:25.425 | INFO     | __main__:run_pipeline_self:79 - %384 - Running pipeline for row_seed: 602 and size_of_unlabelled_data: 0.85 and model: RandomForestClassifier(max_depth=11)
2023-10-22 14:29:27.761 | INFO     | __main__:run_pipeline_self:79 - %385 - Running pipeline for row_seed: 412 and size_of_unlabelled_data: 0.85 and model: RandomForestClassifier(max_depth=11)
2023-10-22 14:29:30.249 | INFO     | __main__:run_pipeline_self:79 - %386 - Running pipeline for row_seed: 214 and size_of_unlabelled_data: 0.85 and model: RandomForestClassifier(max_depth=11)
2023-10-22 14:29:32.659 | INFO     |

2023-10-22 14:29:44.814 | INFO     | __main__:run_pipeline_self:79 - %392 - Running pipeline for row_seed: 42 and size_of_unlabelled_data: 0.9 and model: RandomForestClassifier(max_depth=11)
2023-10-22 14:29:47.030 | INFO     | __main__:run_pipeline_self:79 - %393 - Running pipeline for row_seed: 206 and size_of_unlabelled_data: 0.9 and model: RandomForestClassifier(max_depth=11)
2023-10-22 14:29:49.446 | INFO     | __main__:run_pipeline_self:79 - %394 - Running pipeline for row_seed: 602 and size_of_unlabelled_data: 0.9 and model: RandomForestClassifier(max_depth=11)
2023-10-22 14:29:51.751 | INFO     | __main__:run_pipeline_self:79 - %395 - Running pipeline for row_seed: 412 and size_of_unlabelled_data: 0.9 and model: RandomForestClassifier(max_depth=11)
2023-10-22 14:29:53.637 | INFO     | __main__:run_pipeline_self:79 - %396 - Running pipeline for row_seed: 214 and size_of_unlabelled_data: 0.9 and model: RandomForestClassifier(max_depth=11)
2023-10-22 14:29:56.017 | INFO     | __ma

In [6]:
# mean and standard deviation for all metrics
results_df_with_mean_std = results_df.groupby(['model', 'unlabeled']).agg({'acc': ['mean', 'std'], 'f1': ['mean', 'std'], 'precision': ['mean', 'std'], 'recall': ['mean', 'std'], 'roc': ['mean', 'std']})
results_df_with_mean_std = results_df_with_mean_std.reset_index(level=1)
results_df_with_mean_std.columns = ["_".join(col) for col in results_df_with_mean_std.columns.values]
results_df_with_mean_std.rename(columns={'unlabeled_': 'unlabeled'}, inplace=True)
results_df_with_mean_std.reset_index(inplace=True)
results_df_with_mean_std.columns

Index(['model', 'unlabeled', 'acc_mean', 'acc_std', 'f1_mean', 'f1_std',
       'precision_mean', 'precision_std', 'recall_mean', 'recall_std',
       'roc_mean', 'roc_std'],
      dtype='object')

In [7]:
# Metrics with tabulate
from tabulate import tabulate

print(tabulate(results_df_with_mean_std, headers='keys', tablefmt='psql', showindex=False))

+------------------------+-------------+------------+------------+-----------+------------+------------------+-----------------+---------------+--------------+------------+------------+
| model                  |   unlabeled |   acc_mean |    acc_std |   f1_mean |     f1_std |   precision_mean |   precision_std |   recall_mean |   recall_std |   roc_mean |    roc_std |
|------------------------+-------------+------------+------------+-----------+------------+------------------+-----------------+---------------+--------------+------------+------------|
| DecisionTreeClassifier |        0.1  |     0.855  | 0.00849837 |  0.850007 | 0.00967974 |         0.877062 |      0.0225789  |      0.825703 |   0.0280933  |   0.854883 | 0.0085024  |
| DecisionTreeClassifier |        0.2  |     0.852  | 0.0077746  |  0.845717 | 0.00847551 |         0.879893 |      0.020128   |      0.814859 |   0.0221147  |   0.851852 | 0.00776468 |
| DecisionTreeClassifier |        0.3  |     0.8506 | 0.00760409 |  0.